# OIC-158 TEM Nuclei Segmentation
Packages to import

In [1]:
import napari
import numpy as np
import pyclesperanto as cle
import skimage as sk
from skimage.io import imread, imsave
import pandas as pd
import os
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt

All functions for pipeline

In [ ]:
#Create a 25-pixel wide band along perimeter of nucleus
def band_25_pixels(masks,footprint):
    eroded = [sk.morphology.erosion(mask, footprint=footprint) for mask in masks]
    bands = [masks[i] - eroded[i] for i in range(len(masks))]
    return bands

#Threshold the dense regions within whole image
def norm_and_threhsold(imgs,max_hole=100,min_object=20,match_img=3):
    adpt_norm = [sk.exposure.equalize_adapthist(img) for img in imgs]
    matched_hist = [sk.exposure.match_histograms(img, adpt_norm[match_img]) for img in adpt_norm]
    filtered = []
    for img in matched_hist:
        yen = sk.filters.threshold_yen(img)
        mask = img < yen
        dilate = sk.morphology.dilation(mask)
        filtered.append(sk.morphology.remove_small_objects(sk.morphology.remove_small_holes(dilate,area_threshold=max_hole),min_size=min_object))
    return filtered

#Get dense roi masks within the nuclei and 25-pixel band
def dense_rois_in_nuclei_and_band(dense_roi_masks, band_25_pixels, relabel_nuc_masks):
    dense_roi_in_nuclei = [dense_roi_masks[i] * relabel_nuc_masks[i] for i in range(len(relabel_nuc_masks))]
    dense_roi_in_band = [dense_roi_in_nuclei[i] * band_25_pixels[i] for i in range(len(band_25_pixels))]
    return dense_roi_in_nuclei, dense_roi_in_band

#merge masks and image into array for measurements
def merge_masks_image(imgs,relabel_nuclei_masks,band_25_pixels,dense_roi_in_nuclei, dense_roi_in_band):
    merged_stacks = []
    for i in range(len(imgs)):
        stack = np.stack([imgs[i],relabel_nuclei_masks[i],band_25_pixels[i],dense_roi_in_nuclei[i],dense_roi_in_band[i]],axis=-1)
        merged_stacks.append(stack)
    return merged_stacks

#Get heterochromatin and band areas as lists
def heterochromatin_and_band_areas(dense_roi_in_band, band_25_pixels, scale):
    all_heterochromatin_areas = []
    all_band_areas = []
    all_norm_areas = []
    for i in range(len(band_25_pixels)):
        i_heterochromatin_areas = []
        i_band_areas = []
        i_norm_areas = []
        band_props = sk.measure.regionprops(band_25_pixels[i],spacing=scale)
        heterochromatin_props = sk.measure.regionprops(dense_roi_in_band[i],spacing=scale)
        for j in range(len(band_props)):
            band_area = band_props[j].area
            heterochromatin_area = heterochromatin_props[j].area
            norm_area = heterochromatin_area/band_area
            i_heterochromatin_areas.append(heterochromatin_area)
            i_band_areas.append(band_area)
            i_norm_areas.append(norm_area)
        all_heterochromatin_areas.append(i_heterochromatin_areas)
        all_band_areas.append(i_band_areas)
        all_norm_areas.append(i_norm_areas)
    return all_heterochromatin_areas, all_band_areas, all_norm_areas

#Get measurements for all compartments and merge dataframes together (run in loop for all images)
def get_measurements(merged_stacks,props,scale,all_heterochromatin_areas, all_band_areas, all_norm_areas):
    heterochromatin_areas_df = pd.Series(all_heterochromatin_areas,name='heterochromatin_area_in_band_um^2')
    band_areas_df = pd.Series(all_band_areas, name='25-pixel_band_area_um^2')
    norm_area_df = pd.Series(all_norm_areas, name='norm_area_heterochromatin_in_band')
    nuc_measurements_table = sk.measure.regionprops_table(merged_stacks[:,:,1],merged_stacks,props,spacing=scale)
    nuc_regionprops = sk.measure.regionprops(merged_stacks[:,:,1],spacing=scale)
    circularity = []
    for i in range(len(nuc_regionprops)):
        nuc_area = nuc_regionprops[i].area
        nuc_perimeter = nuc_regionprops[i].perimeter
        circularity.append(4*np.pi*nuc_area/nuc_perimeter**2)
    circularity = pd.Series(circularity,name='nuc_circularity')
    nuc_df = pd.DataFrame.from_dict(nuc_measurements_table)
    nuc_df.rename(columns={'label':'cell_ID','area':'Nuc_area_um^2','intensity_max-0':'Image_intensity_max',
                           'intensity_max-1':'Nuc_ID','intensity_max-2':'Band_ID','intensity_max-3':'Dense_nuc_regions_ID',
                           'intensity_max-4':'Heterochromatin_Band_ID','equivalent_diameter_area':'equivalent_diameter_area_um^2',
                           'perimeter':'perimeter_um'},inplace=True)
    df = pd.concat([nuc_df,circularity,heterochromatin_areas_df,band_areas_df,norm_area_df], axis=1)
    return df

#save images as tiffs and dataframes as csv (run in loop for all images along with get measurements function)
def save_df_and_images(df,merged_stacks,save_loc, img_files):
    image_path = os.path.join(save_loc,'merged_imgs')
    df_path = os.path.join(save_loc,'measurements')
    img_name = os.path.basename(img_files)
    imsave(os.path.join(image_path,'merged_stack_'+img_name[:-4]+'.tif'),merged_stacks,check_contrast=False)
    df.to_csv(os.path.join(df_path,'measurements_'+img_name[:-4]+'.csv'))


Load in data and filter out masks touching border and small labeled areas created during ground truth annotation

In [20]:
img_files = sorted(glob("E:/Fondufe-Mittendorf_Lab/TEM_Images/Images/*.tif"))
mask_files = sorted(glob("E:/Fondufe-Mittendorf_Lab/TEM_Images/Masks/*.tif"))
imgs = list(map(imread,img_files))
imgs = [np.asarray(img,dtype=np.uint16) for img in imgs] #make sure all imgs are np compatible arrays
nuclei_masks = list(map(imread,mask_files))
nuclei_masks = [np.asarray(mask,dtype=np.uint16) for mask in nuclei_masks] #make sure all label images are compatible arrays
filtered_nuclei_masks = [sk.segmentation.clear_border(mask) for mask in nuclei_masks]
relabel_nuclei_masks = [sk.measure.label(mask) for mask in filtered_nuclei_masks]
remove_small_objects = [sk.morphology.remove_small_objects(mask, 50) for mask in relabel_nuclei_masks]

E:\Temp\ipykernel_29820\2255796120.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  remove_small_objects = [sk.morphology.remove_small_objects(mask, 50) for mask in relabel_nuclei_masks]


In [22]:
footprint = sk.morphology.disk(25)
bands_25 = band_25_pixels(masks=remove_small_objects,footprint=footprint)
dense_roi_masks = norm_and_threhsold(imgs)
dense_roi_in_nuclei, dense_roi_in_band = dense_rois_in_nuclei_and_band(dense_roi_masks=dense_roi_masks,band_25_pixels=bands_25,relabel_nuc_masks=remove_small_objects)
merged_stacks = merge_masks_image(imgs,remove_small_objects,bands_25,dense_roi_in_nuclei,dense_roi_in_band)
heterochromatin_areas = heterochromatin_area(dense_roi_in_band,bands_25,scale=[0.0064,0.0064])


c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\filters\thresholding.py:466: RuntimeWarning: divide by zero encountered in log
  crit = np.log(((P1_sq[:-1] * P2_sq[1:]) ** -1) * (P1[:-1] * (1.0 - P1[:-1])) ** 2)


In [25]:
heterochromatin_areas = heterochromatin_area(dense_roi_in_band,bands_25,scale=[0.0064,0.0064])

In [26]:
#collecting measurements for all images
props = ['label', 'area', 'eccentricity', 'equivalent_diameter_area','extent','perimeter','solidity','intensity_max']
scale = [0.0064,0.0064] #in um
save_loc = 'Results/'
for i in range(len(imgs)):
    swapped_axis = np.moveaxis(merged_stacks[i],-1,0)
    df = get_measurements(merged_stacks[i],props=props,scale=scale,heterochromatin_areas=heterochromatin_areas[i])
    save_df_and_images(df,swapped_axis,save_loc=save_loc,img_files=img_files[i])

### testing cells for creating band

In [ ]:
#erode objects to get a thin outer border (disc size 25 is better option)
footprint_50 = sk.morphology.disk(50)
eroded = sk.morphology.erosion(test_mask,footprint_50)
band_50 = test_mask - eroded
# viewer = napari.view_image(relabeled)
# viewer.add_labels(eroded)
# viewer.add_labels(band_50)

In [ ]:
test_mask = filtered_masks[11]
footprint_25 = sk.morphology.disk(25)
eroded = sk.morphology.erosion(test_mask,footprint_25)
band_25 = test_mask - eroded
# viewer = napari.view_image(imgs[11])
# viewer.add_labels(eroded)
# viewer.add_labels(band_25, name='25')

## Thresholding with Yen

In [ ]:
#footprint = sk.morphology.disk(2)
adpt_norm = [sk.exposure.equalize_adapthist(img) for img in imgs]
matched_hist = [sk.exposure.match_histograms(img, adpt_norm[3]) for img in adpt_norm]
filtered = []
for img in matched_hist:
    yen = sk.filters.threshold_yen(img)
    mask = img < yen
    dilate = sk.morphology.dilation(mask)
    filtered.append(sk.morphology.remove_small_objects(sk.morphology.remove_small_holes(dilate,area_threshold=100),min_size=20))

In [ ]:
stack = np.stack(filtered)
viewer = napari.view_labels(stack)

## Get area of band covered by dense signal

In [ ]:
mask_dense_nuclei = [filtered[i] * relabel_masks[i] for i in range(len(filtered))]
#mask_dense_nuclei_stack = np.stack(mask_dense_nuclei)
dense_band = [mask_dense_nuclei[i] * bands[i] for i in range(len(mask_dense_nuclei))]
#dense_band_stack = np.stack(dense_band)



Merge the image and all masks into arrays

In [ ]:
merged_stacks = []
for i in range(len(imgs)):
    stack = np.stack([imgs[i],relabel_masks[i],bands[i],mask_dense_nuclei[i],dense_band[i]],axis=-1)
    merged_stacks.append(stack)

In [ ]:
merged_stacks[0].shape

## Get measurements

In [ ]:
props = ['label', 'area', 'eccentricity', 'equivalent_diameter_area','extent','perimeter','solidity','intensity_max',]
scale = [0.0064,0.0064]

In [ ]:
def heterochromatin_area(heterochromatin_band, bands, scale):
    areas = []
    for i in range(1,np.max(bands)):
        band = bands == i
        mask = band * heterochromatin_band
        area = (np.count_nonzero(mask) * scale[0] * scale[1])
        areas.append(area)
    areas_df = pd.Series(areas, name='heterochromatin_area_um^2')
    return areas, areas_df


In [ ]:
areas, areas_df = heterochromatin_area(heterochromatin_band=dense_band[24],bands=bands[24],scale=scale)
areas

In [ ]:
def get_measurements(img_stack,props,scale,heterochromatin_areas, heterochromatin_area_df):
    nuc_measurements_table = sk.measure.regionprops_table(img_stack[:,:,1],img_stack,props,spacing=scale)
    nuc_regionprops = sk.measure.regionprops(img_stack[:,:,1],spacing=scale)
    band_25_measurements= sk.measure.regionprops_table(img_stack[:,:,2],properties=('label','area'),spacing=scale)
    band_25_df = pd.DataFrame.from_dict(band_25_measurements)
    band_25_df = band_25_df.rename(columns={'label':'Band_ID','area':'area_um^2'})
    circularity = []
    for i in range(len(nuc_regionprops)):
        nuc_area = nuc_regionprops[i].area
        nuc_perimeter = nuc_regionprops[i].perimeter
        circularity.append(4*np.pi*nuc_area/nuc_perimeter**2)
    circularity = pd.Series(circularity,name='nuc_circularity')
    percent_band_covered = []
    band_25_regionprops= sk.measure.regionprops(img_stack[:,:,2],spacing=scale)
    for i in range(len(heterochromatin_areas)):
        band_area = band_25_regionprops[i].area
        percent_covered = heterochromatin_areas[i]/band_area
        percent_band_covered.append(percent_covered)
    percent_band_covered = pd.Series(percent_band_covered, name='norm_area_heterochromatin_band')
    nuc_df = pd.DataFrame.from_dict(nuc_measurements_table)
    nuc_df.rename(columns={'label':'cell_ID','area':'Nuc_area_um^2','intensity_max-0':'Image_intensity_max',
                           'intensity_max-1':'Nuc_ID','intensity_max-2':'Band_ID','intensity_max-3':'Dense_nuc_regions_ID',
                           'intensity_max-4':'Heterochromatin_Band_ID','equivalent_diameter_area':'equivalent_diameter_area_um^2',
                           'perimeter':'perimeter_um'},inplace=True)
    df = pd.concat([nuc_df,circularity,heterochromatin_area_df,percent_band_covered,band_25_df], axis=1)
    return df


In [ ]:
test_measurements = get_measurements(merged_stacks[24],props,scale,heterochromatin_areas=areas,heterochromatin_area_df=areas_df)

In [ ]:
test_measurements.head()

In [ ]:
viewer = napari.view_image(images_stack, name='img')
viewer.add_labels(label_stack, name='nuclei mask')
viewer.add_labels(bands_stack,name='25px band')
viewer.add_labels(mask_dense_nuclei_stack,name='dense areas in nuc')
viewer.add_labels(dense_band_stack, name='heterochromatin/')


### Multi-Otsu testing
4 classes with preprocessing of gaussian sigma=2; use with ground truth masks

Use to measure the dense heterochromatin along the membrane and the nucleoli within the nuclei

In [ ]:
#find threshold or approach that works well for segmenting the dense areas
#multi-otsu
mask = test_mask == 1
gauss = sk.filters.gaussian(imgs[11], sigma=2)
masked_img = mask * gauss
o1,o2,o3 = sk.filters.threshold_multiotsu(image=masked_img, classes=4)



In [ ]:
otsu1 = (masked_img < o1)*mask
otsu2 = (masked_img < o2)*mask
otsu3 = (masked_img < o3)*mask

In [ ]:
viewer = napari.view_image(imgs[11], name='image')
viewer.add_labels(otsu1, name='otsu1')
viewer.add_labels(otsu2, name='otsu2')
viewer.add_labels(otsu3, name='otsu3')

In [ ]:
test_img = imgs[24]
test_img = np.asarray(test_img,dtype=np.uint16)

## Test histogram norm with histogram matching

In [ ]:
# Correct uneven illumination

adpt_norm = sk.exposure.equalize_adapthist(test_img)
norm = sk.exposure.equalize_hist(test_img)
# viewer = napari.view_image(test_img)
# viewer.add_image(adpt_norm, name='adapt norm')
# viewer.add_image(norm, name='norm')

In [ ]:
# add in gamma and contrast adjustment to enhance dense regions; didn't make a large difference for thresholding
gamma_adjust = sk.exposure.adjust_gamma(adpt_norm, gamma=1.5)
viewer = napari.view_image(adpt_norm)
viewer.add_image(gamma_adjust, name='gamma')

In [ ]:
# some images fail on ostu multiclass, testing if matching histogram to images that have successful ostu works


## Testing Sobel and Canny for segmenting the nuclei

### Canny Testing
Close! But not enough connectivity with the lines from canny


In [ ]:
footprint = sk.morphology.disk(6)
top_hat = sk.morphology.black_tophat(adpt_norm)
viewer = napari.view_image(top_hat)

In [ ]:
edges = sk.feature.canny(gamma_adjust, sigma=2.4)
viewer = napari.view_image(adpt_norm)
viewer.add_image(edges, name='edges')

## Sobel Testing
Not the best approach for TEM images. Not enough contrast in the staining for sobel to find the features of interest

In [ ]:
edges = sk.filters.sobel(adpt_norm)
viewer = napari.view_image(adpt_norm)
viewer.add_image(edges, name='edges')

## Multi-Otsu on histogram equalized images
Use second otsu value

In [ ]:
#test image and masks
test_mask = filtered_masks[24]
test_img = np.asarray(imgs[24],dtype=np.uint16)
adpt_norm = sk.exposure.equalize_adapthist(test_img)

In [ ]:
fig, ax = sk.filters.try_all_threshold(adpt_norm)

In [ ]:
#trying yen threshold
mask = test_mask == 1
gauss = sk.filters.gaussian(adpt_norm, sigma=1)
masked_img = mask * gauss
yen = sk.filters.threshold_yen(image=gauss)
yen_mask = adpt_norm < yen
viewer = napari.view_image(adpt_norm)
viewer.add_labels(yen_mask)

In [ ]:
#trying triangle threshold
mask = test_mask == 1
gauss = sk.filters.gaussian(adpt_norm, sigma=1)
masked_img = mask * gauss
triangle = sk.filters.threshold_triangle(image=gauss)
triangle_mask = adpt_norm < triangle
viewer = napari.view_image(adpt_norm)
viewer.add_labels(triangle_mask)

In [ ]:
mask = test_mask == 1
gauss = sk.filters.gaussian(adpt_norm, sigma=1)
masked_img = mask * gauss
o1,o2,o3 = sk.filters.threshold_multiotsu(image=masked_img, classes=4)
# otsu1 = (masked_img < o1)*mask
otsu2 = (masked_img < o2)*mask
# otsu3 = (masked_img < o3)*mask
viewer = napari.view_image(adpt_norm, name='image')
#viewer.add_labels(otsu1, name='otsu1')
viewer.add_labels(otsu2, name='otsu2')
#viewer.add_labels(otsu3, name='otsu3')

In [ ]:
labeled_img = sk.morphology.label(otsu2)
props = ['label','area']
region_props = sk.measure.regionprops_table(labeled_img,properties=props)
df = pd.DataFrame.from_dict(region_props)
areas = np.asarray(df['area'])

In [ ]:
fig, ax = plt.subplots()
ax.hist(areas,bins=100,range=[0,2000])
ax.set_yscale('log')
plt.show()

In [ ]:
#filter out small objects and fill holes
filtered = sk.morphology.remove_small_objects(sk.morphology.remove_small_holes(otsu2,area_threshold=10000),min_size=500)
viewer = napari.view_image(adpt_norm, name='img')
viewer.add_labels(filtered, name='labels')
viewer.add_labels(otsu2, name='otsu')

In [ ]:
viewer.add_labels(band_25,name='band')

## Adding in measurements of heterochromatin in band and nucleoli in nuclei

In [ ]:
view